In [217]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns',None)

data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submission.csv')

# 시간 재정렬

In [3]:
data= data.set_index('ID')

In [228]:
data[['Estimated_Departure_Time', 'Estimated_Arrival_Time']] = data[['Estimated_Departure_Time', 'Estimated_Arrival_Time']].fillna(0)
data[['Estimated_Departure_Time', 'Estimated_Arrival_Time']] = data[['Estimated_Departure_Time', 'Estimated_Arrival_Time']].astype(int)
data[['Estimated_Departure_Time', 'Estimated_Arrival_Time']] = data[['Estimated_Departure_Time', 'Estimated_Arrival_Time']].astype(str)
data['Estimated_Departure_Time'] = data['Estimated_Departure_Time'].str.pad(width=4, side='left', fillchar='0')
data['Estimated_Arrival_Time'] = data['Estimated_Arrival_Time'].str.pad(width=4, side='left', fillchar='0')

data['h'] = [data['Estimated_Departure_Time'][i][:2] for i in range(len(data))]
data['m'] = [data['Estimated_Departure_Time'][i][2:] for i in range(len(data))]
data['H'] = [data['Estimated_Arrival_Time'][i][:2] for i in range(len(data))]
data['M'] = [data['Estimated_Arrival_Time'][i][2:] for i in range(len(data))]

data['h'] = data['h'].astype(int)
data['m'] = data['m'].astype(int)
data['H'] = data['H'].astype(int)
data['M'] = data['M'].astype(int)

data = data[~((data['Estimated_Departure_Time'] == '0000') & (data['Estimated_Arrival_Time'] == '0000'))]
data.head()

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,h,m,H,M
1,TRAIN_000001,8,15,0740,1024,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN,7,40,10,24
2,TRAIN_000002,9,6,1610,1805,0,0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN,16,10,18,5
3,TRAIN_000003,7,10,0905,1735,0,0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN,9,5,17,35
4,TRAIN_000004,1,11,0900,1019,0,0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN,9,0,10,19
5,TRAIN_000005,4,13,1545,0000,0,0,EWR,11618,NaN,DCA,11278,Virginia,199.0,Republic Airlines,UA,20452.0,N657RW,Not_Delayed,15,45,0,0


In [232]:
# 시간 계산
nzdata = data[~((data['Estimated_Departure_Time'] == '0000') | (data['Estimated_Arrival_Time'] == '0000'))]

# hour
nzdata['ht'] = nzdata['H'] - nzdata['h']
nzdata.loc[nzdata['ht'] < 0, 'ht'] += 24

nzdata['mt'] = nzdata['M'] - nzdata['m']
nzdata.loc[nzdata['mt'] < 0, 'ht'] -= 1
nzdata.loc[nzdata['ht'] < 0, 'ht'] += 24
nzdata.loc[nzdata['mt'] < 0, 'mt'] += 60

# datetime.strptime(nzdata['Estimated_Departure_Time'], '%H%M')

In [235]:
nzdata['Estimated_Arrival_Time'].isna().sum()

0

In [84]:
from datetime import timedelta, datetime

# datetime(datetime.minute,   (timedelta(hours=10, minutes= 10) - timedelta(hours=9, minutes= 10)) )

dum = timedelta(hours=24, minutes= 4) - timedelta(hours=9, minutes= 10)

dum.seconds//3600
(dum.seconds % 3600) // 60 

54

In [63]:
from datetime import timedelta, datetime

def diff_time(row):
    dep_time = datetime.strptime(str(row['Estimated_Departure_Time']), '%H%M').time()
    print(row['Estimated_Arrival_Time'])
    arr_time = datetime.strptime(str(row['Estimated_Arrival_Time']), '%H%M').time()

    diff = datetime.combine(datetime.min, arr_time) - datetime.combine(datetime.min, dep_time)
    return diff.seconds // 3600, (diff.seconds // 60) % 60
nzdata[['test', 'test1']] = nzdata[['Estimated_Departure_Time', 'Estimated_Arrival_Time']].apply(diff_time, axis=1)


1024
1805
1735
1019
1903
1955
1550
0838
1844
0748
1145
0706
0855
1532
1412
1624
1258
2145
2155
1150
2259
0805
2217
1742
0847
1014
0545
0748
0810
0705
1936
0555
1607
2210
1543
1130
1506
0728
1846
1810
2013
1423
2020
1140
1640
1840
1023
2109
1900
0808
0704
1258
1935
1840
1743
1419
1925
2040
2200
0520
1105
1140
1507
2325
1652
1730
2130
1004
1351
0950
1312
1345
1454
1200
1205
0923
0730
1558
1124
1713
1220
1605
1935
2159
2002
1915
0718
1700
1815
1558
2325
1010
2104
1247
2347
2341
2325
1645
2335
1100
2115
0505
1655
1427
0819
1625
1110
0829
1735
1720
1348
0754
0745
1315
1415
1511
0855
0735
2255
2210
2058
1855
1818
0838
0843
1222
1730
0850
1528
1850
1320
0559
1920
0649
1431
1815
1911
2000
1127
2249
1026
2328
1725
0808
1005
1037
1925
0915
0855
0721
1748
1748
2255
1125
2129
2130
2127
0852
0729
1143
0011
1411
1102
1418
1910
1559
1103
1104
2122
1806
1442
2306
1655
2359
0703
2249
1553
1855
2110
0904
0936
1120
1720
1935
1143
1829
1732
1911
1346
1955
2225
0559
1951
1645
2225
1245
2316
1429
0900
1915


ValueError: unconverted data remains: 0

In [174]:
data['Day_of_Month'].value_counts()

21    33316
18    33234
19    33174
12    33115
16    32958
26    32955
11    32758
20    32695
15    32683
10    32664
17    32656
9     32643
8     32617
25    32603
14    32481
24    32416
13    32392
27    32342
7     32252
6     32201
23    32139
22    32083
2     31938
28    31923
5     31916
4     31837
1     31818
3     31669
30    30763
29    30199
31    17872
Name: Day_of_Month, dtype: int64

In [27]:
from datetime import datetime
(datetime.strptime('0730', '%H%M') - datetime.strptime('1050', '%H%M'))

datetime.timedelta(days=-1, seconds=74400)

In [188]:
data['Airline'].unique()

data[(data['Airline'] == 'SkyWest Airlines Inc.') & (data['Destination_Airport'] == 'BGM') & (data['Origin_Airport'] == 'DTW') & (data['Estimated_Departure_Time'] == '1550')]

,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,Estimated_Departure_Time_H,Estimated_Departure_Time_M,Estimated_Arrival_Time_H,Estimated_Arrival_Time_M
ID,,,,,,,,,,,,,,,,,,,,,,
TRAIN_186902,3,31,1550,0000,0,0,DTW,11433,Michigan,BGM,10577,New York,378.0,SkyWest Airlines Inc.,DL,20304.0,N455CA,NaN,15,50,0,0
TRAIN_191685,3,29,1550,1715,0,0,DTW,11433,Michigan,BGM,10577,NaN,378.0,SkyWest Airlines Inc.,DL,20304.0,N923EV,Not_Delayed,15,50,17,15


In [206]:


def diff_time( df):

    time = timedelta(hours=df['H'], minutes=df['M']) - timedelta(hours=df['h'], minutes=df['m'])
    # print(time)
    time = round(time.total_seconds())
    
    hours = time // 3600
    minutes = (time % 3600) // 60
    
    if hours < 0:
        hours += 24
    
    # if minutes < 0:
    #     minutes += 60
    return hours, minutes 
lst = []
a = nzdata.apply(diff_time, axis=1, result_type='expand')

In [267]:
def to_time(time_list):
    Time = pd.Series(time_list).astype(str).str.zfill(4)
    Time = Time.replace('2400','0000')
    return [datetime.strptime(i, '%H%M').strftime("%H:%M") if i != '0nan' else np.NaN for i in Time]

time_list = [i if str(i) == 'nan' else str(int(i))  for i in data['Estimated_Departure_Time'] ]
time_list1 = [i if str(i) == 'nan' else str(int(i))  for i in data['Estimated_Arrival_Time'] ]
data['Estimated_Departure_Time_HH:MM'] = to_time(time_list)
data['Estimated_Arrival_Time_HH:MM'] = to_time(time_list1)

In [271]:
data[data['Estimated_Arrival_Time_HH:MM'] < '2:00']

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,h,m,H,M,Estimated_Departure_Time_HH:MM,Estimated_Arrival_Time_HH:MM
1,TRAIN_000001,8,15,0740,1024,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN,7,40,10,24,07:40,10:24
2,TRAIN_000002,9,6,1610,1805,0,0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN,16,10,18,5,16:10,18:05
3,TRAIN_000003,7,10,0905,1735,0,0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN,9,5,17,35,09:05,17:35
4,TRAIN_000004,1,11,0900,1019,0,0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN,9,0,10,19,09:00,10:19
5,TRAIN_000005,4,13,1545,0000,0,0,EWR,11618,NaN,DCA,11278,Virginia,199.0,Republic Airlines,UA,20452.0,N657RW,Not_Delayed,15,45,0,0,15:45,00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,0936,1243,0,0,ORD,13930,NaN,PHL,14100,NaN,678.0,United Air Lines Inc.,UA,19977.0,N477UA,NaN,9,36,12,43,09:36,12:43
999996,TRAIN_999996,5,30,0920,1028,0,0,FAR,11637,NaN,MSP,13487,Minnesota,223.0,SkyWest Airlines Inc.,DL,NaN,N439SW,NaN,9,20,10,28,09:20,10:28
999997,TRAIN_999997,6,28,0800,1340,0,0,OAK,13796,NaN,HOU,12191,Texas,1642.0,Southwest Airlines Co.,WN,19393.0,N230WN,NaN,8,0,13,40,08:00,13:40
999998,TRAIN_999998,9,27,1613,1824,0,0,BNA,10693,Tennessee,ATL,10397,NaN,214.0,Delta Air Lines Inc.,DL,19790.0,N968DL,NaN,16,13,18,24,16:13,18:24


In [281]:
from tqdm import tqdm
pd.set_option('mode.chained_assignment',  None)
class Processing:
    def __init__(self, x):
        self.x = x.set_index('ID')
        self.nzdata = nzdata
        
        # 시간 패딩
        self.x[['Estimated_Departure_Time', 'Estimated_Arrival_Time']] = self.x[['Estimated_Departure_Time', 'Estimated_Arrival_Time']].fillna(0)
        self.x[['Estimated_Departure_Time', 'Estimated_Arrival_Time']] = self.x[['Estimated_Departure_Time', 'Estimated_Arrival_Time']].astype(int)
        self.x[['Estimated_Departure_Time', 'Estimated_Arrival_Time']] = self.x[['Estimated_Departure_Time', 'Estimated_Arrival_Time']].astype(str)
        self.x['Estimated_Departure_Time'] = self.x['Estimated_Departure_Time'].str.pad(width=4, side='left', fillchar='0')
        self.x['Estimated_Arrival_Time'] = self.x['Estimated_Arrival_Time'].str.pad(width=4, side='left', fillchar='0')
        
        # 시간 둘 다 NULL값인 경우 제거함.
        self.x = self.x[~((self.x['Estimated_Departure_Time'] == '0000') & (self.x['Estimated_Arrival_Time'] == '0000'))]

    def diff_time(self, df):
    
        time = timedelta(hours=df['H'], minutes=df['M']) - timedelta(hours=df['h'], minutes=df['m'])
        # print(time)
        time = round(time.total_seconds())
        
        hours = time // 3600
        minutes = (time % 3600) // 60
        
        if hours < 0:
            hours += 24
        
        # if minutes < 0:
        #     minutes += 60
        return hours, minutes 
        
    
    
    def make_noise_zero_data(self):
        # 시간 NULL값 없는 경우만 뽑음
        # nzdata = self.x[~(self.x(['Estimated_Departure_Time'] == '0000') | (self.x['Estimated_Arrival_Time'] == '0000'))]
        nzdata = self.x[~((self.x['Estimated_Departure_Time'] == '0000') | (self.x['Estimated_Arrival_Time'] == '0000'))]
        nzdata = self.make_time_feature(nzdata)
        time = nzdata.apply(self.diff_time, axis=1, result_type='expand')
        nzdata['diff_hour'] = time[0]
        nzdata['diff_minute'] = time[1]
        
        return nzdata
        

    
    def make_time_feature(self, data):
        data['h'] = [data['Estimated_Departure_Time'][i][:2] for i in range(len(data))]
        data['m'] = [data['Estimated_Departure_Time'][i][2:] for i in range(len(data))]
        data['H'] = [data['Estimated_Arrival_Time'][i][:2] for i in range(len(data))]
        data['M'] = [data['Estimated_Arrival_Time'][i][2:] for i in range(len(data))]

        data['h'] = data['h'].astype(int)
        data['m'] = data['m'].astype(int)
        data['H'] = data['H'].astype(int)
        data['M'] = data['M'].astype(int)

        return data
    
    def fill_time(self):
        print('시간 전처리')
        x = self.make_time_feature(self.x)
        check = self.make_noise_zero_data()
        
        # 출발시간
        print('출발시간')
        idx = x[x['Estimated_Departure_Time'] == '0000'].index
        for i in tqdm(idx):
            at = x.loc[i, 'Estimated_Arrival_Time']
            dt = x.loc[i, 'Estimated_Departure_Time']
            oa = x.loc[i, 'Origin_Airport']
            da = x.loc[i, 'Destination_Airport']
            al = x.loc[i, 'Airline']
            
            check_data = check[(check['Origin_Airport'] == oa) & (check['Destination_Airport'] == da) & (check['Airline'] == al)]
            # print(check_data)
            
            # 동일 시간 존재 시 동일한 수로 대체 (복수 존재 시 제일 많은걸로)
            insert_time = check_data.loc[check_data['Estimated_Arrival_Time'] == at, 'Estimated_Departure_Time']
            if len(insert_time) != 0:
                try:
                    x.loc[i, 'Estimated_Departure_Time'] = insert_time.value_counts().index.drop('0000')[0]
                
                except:

                    
                    x.loc[i, 'Estimated_Departure_Time'] = insert_time.value_counts().index[0]
                    
                x.loc[i, 'h'] = int(x.loc[i, 'Estimated_Departure_Time'][:2])
                x.loc[i, 'm'] = int(x.loc[i, 'Estimated_Departure_Time'][2:])
                
            # 가장 빈도수 높은 경과시간으로 계산하여 대체
            elif len(insert_time) == 0:
                

                try:
                    x.loc[i, 'Estimated_Departure_Time'] = check_data['Estimated_Departure_Time'].value_counts().index.drop('0000')[0]
                
                except:
                    print(at, dt, oa, da, al)
                    print(insert_time.value_counts())
                    print(insert_time.value_counts().index[0])
                    x.loc[i, 'Estimated_Departure_Time'] = check_data['Estimated_Departure_Time'].value_counts().index[0]
                    
                x.loc[i, 'H'] = int(x.loc[i, 'Estimated_Departure_Time'][:2])
                x.loc[i, 'M'] = int(x.loc[i, 'Estimated_Departure_Time'][2:])
            
            # 동일 시간 없을 시 평균 경과시간으로 대체
            # elif len(insert_time) == 0:
            #     ht = round(check_data['diff_hour'].mean())
            #     mt = round(check_data['diff_minute'].mean())
                
            #     insert_time =  timedelta(hours=x.loc[i, 'H'], minutes=x.loc[i, 'M']) - timedelta(hours=ht, minutes=mt)
            #     insert_time = round(insert_time.total_seconds())
            #     hours = insert_time // 3600
            #     minutes = (insert_time % 3600) // 60
                
            #     if hours < 0:
            #         hours += 24

            #     x.loc[i, 'h'] = hours
            #     x.loc[i, 'm'] = minutes
                
                
        # 도착시간
        print('도착시간')
        idx = x[x['Estimated_Arrival_Time'] == '0000'].index
        for i in tqdm(idx):
            at = x.loc[i, 'Estimated_Arrival_Time']
            dt = x.loc[i, 'Estimated_Departure_Time']
            oa = x.loc[i, 'Origin_Airport']
            da = x.loc[i, 'Destination_Airport']
            al = x.loc[i, 'Airline']
            
            check_data = check[(check['Origin_Airport'] == oa) & (check['Destination_Airport'] == da) & (check['Airline'] == al)]
            # 동일 시간 존재 시 동일한 수로 대체 (복수 존재 시 제일 많은걸로)
            insert_time = check_data.loc[check_data['Estimated_Departure_Time'] == dt, 'Estimated_Arrival_Time']
            if len(insert_time) != 0:
                try:
                    x.loc[i, 'Estimated_Arrival_Time'] = insert_time.value_counts().index.drop('0000')[0]
                
                except:
                    x.loc[i, 'Estimated_Arrival_Time'] = insert_time.value_counts().index[0]
                    
                x.loc[i, 'H'] = int(x.loc[i, 'Estimated_Arrival_Time'][:2])
                x.loc[i, 'M'] = int(x.loc[i, 'Estimated_Arrival_Time'][2:])
            
            # 가장 빈도수 높은 경과시간으로 계산하여 대체
            elif len(insert_time) == 0:
                try:
                    x.loc[i, 'Estimated_Arrival_Time'] = check_data['Estimated_Arrival_Time'].value_counts().index.drop('0000')[0]
                
                except:
                    x.loc[i, 'Estimated_Arrival_Time'] = check_data['Estimated_Arrival_Time'].value_counts().index[0]
                    
                x.loc[i, 'H'] = int(x.loc[i, 'Estimated_Arrival_Time'][:2])
                x.loc[i, 'M'] = int(x.loc[i, 'Estimated_Arrival_Time'][2:])
            
            
            
            
            # 동일 시간 없을 시 평균 경과시간으로 대체
            # elif len(insert_time) == 0:
            #     ht = round(check_data['diff_hour'].mean())
            #     mt = round(check_data['diff_minute'].mean())
                
            #     # 23 50 / 1 40 -> 1 50
                
            #     insert_time =  timedelta(hours=x.loc[i, 'h'], minutes=x.loc[i, 'm']) + timedelta(hours=ht, minutes=mt)
            #     insert_time = round(insert_time.total_seconds())
                    
            #     hours = insert_time // 3600
            #     minutes = (insert_time % 3600) // 60
                
            #     if hours > 24:
            #         hours -= 24

            #     x.loc[i, 'h'] = hours
            #     x.loc[i, 'm'] = minutes
                
        return x
    
    
                
                
            
        

In [282]:
test = Processing(data)

t = test.fill_time()


시간 전처리
출발시간


  0%|          | 2/97331 [00:00<5:47:16,  4.67it/s]


1446 0000 IAH ORD nan
Series([], Name: Estimated_Departure_Time, dtype: int64)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [276]:
t

NameError: name 't' is not defined

In [259]:
dum = data.set_index('ID')

dum.index.to_numpy()[0]

'TRAIN_000001'

In [264]:
data[(data['Origin_State'] == 'Kansas') & (data['Destination_State'] == 'Kansas')]

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,h,m,H,M
14993,TRAIN_014993,9,2,2255,2334,0,0,HYS,12255,Kansas,SLN,14877,Kansas,88.0,SkyWest Airlines Inc.,UA,NaN,N701BR,NaN,22,55,23,34
17143,TRAIN_017143,7,13,0540,0621,0,0,SLN,14877,Kansas,HYS,12255,Kansas,88.0,SkyWest Airlines Inc.,UA,20304.0,N973SW,NaN,5,40,6,21
22401,TRAIN_022401,6,10,0530,0610,0,0,SLN,14877,Kansas,HYS,12255,Kansas,88.0,NaN,UA,20304.0,N980SW,NaN,5,30,6,10
27858,TRAIN_027858,11,16,2305,2357,0,0,HYS,12255,Kansas,SLN,14877,Kansas,88.0,SkyWest Airlines Inc.,UA,20304.0,N925SW,NaN,23,5,23,57
30629,TRAIN_030629,4,20,0545,0620,0,0,SLN,14877,Kansas,HYS,12255,Kansas,88.0,SkyWest Airlines Inc.,UA,20304.0,N930SW,NaN,5,45,6,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975572,TRAIN_975572,12,30,2246,2336,0,0,HYS,12255,Kansas,SLN,14877,Kansas,88.0,SkyWest Airlines Inc.,UA,20304.0,N466SW,Not_Delayed,22,46,23,36
980215,TRAIN_980215,10,11,2255,2334,0,0,HYS,12255,Kansas,SLN,14877,Kansas,88.0,SkyWest Airlines Inc.,UA,20304.0,N919SW,NaN,22,55,23,34
982685,TRAIN_982685,12,27,1510,1600,0,0,HYS,12255,Kansas,SLN,14877,Kansas,88.0,SkyWest Airlines Inc.,UA,20304.0,N967SW,NaN,15,10,16,0
989869,TRAIN_989869,7,23,2245,0000,0,0,HYS,12255,Kansas,SLN,14877,Kansas,88.0,SkyWest Airlines Inc.,UA,20304.0,N932SW,NaN,22,45,0,0


In [125]:
data[(data['Origin_Airport'] == 'OKC') & (data['Destination_Airport'] == 'HOU') & (data['Month'] == 4)]
data[(data['Origin_Airport'] == 'OKC') & (data['Destination_Airport'] == 'HOU')]

,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,Estimated_Departure_Time_H,Estimated_Departure_Time_M,Estimated_Arrival_Time_H,Estimated_Arrival_Time_M
ID,,,,,,,,,,,,,,,,,,,,,,
TRAIN_000000,4,15,0000,0000,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN,0,0,0,0
TRAIN_000827,7,16,0550,0715,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N733SA,NaN,5,50,7,15
TRAIN_001452,1,13,1405,1535,0,0,OKC,13851,Oklahoma,HOU,12191,NaN,419.0,Southwest Airlines Co.,WN,19393.0,N204WN,NaN,14,5,15,35
TRAIN_004015,5,17,1050,1215,0,0,OKC,13851,Oklahoma,HOU,12191,NaN,419.0,Southwest Airlines Co.,WN,19393.0,N246LV,NaN,10,50,12,15
TRAIN_009325,8,14,0600,0725,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7706A,NaN,6,0,7,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TRAIN_991185,1,10,1915,2045,0,0,OKC,13851,NaN,HOU,12191,Texas,419.0,NaN,WN,19393.0,N8684F,Not_Delayed,19,15,20,45
TRAIN_991368,8,14,1450,1610,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N8621A,NaN,14,50,16,10
TRAIN_993122,1,14,0600,0730,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N765SW,NaN,6,0,7,30


In [124]:
data[data['Estimated_Departure_Time'] == '0000'].index.unique().shape

(109019,)

# 데이터 분리

In [169]:
data['Delay'].unique() # nan, 'Not_Delayed', 'Delayed'
nd = data.loc[data['Delay'].isin(['Not_Delayed'])]
d = data.loc[data['Delay'].isin(['Delayed'])]
nan = data.loc[~data['Delay'].isin(['Not_Delayed', 'Delayed'])]

In [177]:
data['Tail_Number'].unique().shape

(6430,)

# NULL값 처리


In [97]:
data[(data['Carrier_Code(IATA)'] == 'UA') & (data['Airline'] == 'United Air Lines Inc.')] # 코드는 맞출 수 있을거같

,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,Estimated_Departure_Time_H,Estimated_Departure_Time_M,Estimated_Arrival_Time_H,Estimated_Arrival_Time_M
ID,,,,,,,,,,,,,,,,,,,,,,
TRAIN_000003,7,10,0905,1735,0,0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN,9,5,17,35
TRAIN_000023,10,18,1750,2259,0,0,LAX,12892,California,MCI,13198,Missouri,1363.0,Delta Air Lines Inc.,DL,19790.0,N355NW,NaN,17,50,22,59
TRAIN_000033,4,12,2158,0545,0,0,LAX,12892,California,FLL,11697,Florida,2343.0,Alaska Airlines Inc.,AS,19930.0,N621VA,NaN,21,58,5,45
TRAIN_000058,3,28,1129,1258,0,0,LAX,12892,California,PHX,14107,Arizona,370.0,American Airlines Inc.,AA,19805.0,N204UW,NaN,11,29,12,58
TRAIN_000108,1,21,1950,2104,0,0,LAX,12892,California,LAS,12889,NaN,236.0,American Airlines Inc.,AA,19805.0,N561UW,NaN,19,50,21,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TRAIN_999828,6,10,1935,0000,0,0,LAX,12892,California,OAK,13796,California,337.0,Southwest Airlines Co.,WN,19393.0,N412WN,NaN,19,35,0,0
TRAIN_999871,12,2,0815,1625,0,0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,19977.0,N598UA,NaN,8,15,16,25
TRAIN_999938,4,27,1225,2046,0,0,LAX,12892,California,JFK,12478,New York,2475.0,Delta Air Lines Inc.,DL,19790.0,N171DN,NaN,12,25,20,46
